In [23]:
import tensorflow as tf
from keras import layers, models

def create_cnn_sdd_model(input_shape=(224, 224, 3), num_classes=61):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(256, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(1024))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model


In [24]:
from keras import optimizers

model = create_cnn_sdd_model()

opt = optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 224, 224, 32)      896       
                                                                 
 batch_normalization_35 (Bat  (None, 224, 224, 32)     128       
 chNormalization)                                                
                                                                 
 activation_35 (Activation)  (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 batch_normalization_36 (Bat  (None, 112, 112, 64)    

In [25]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = 'data/WIDERFace/WIDER_train/'
validation_dir = 'data/WIDERFace/WIDER_val/'
test_dir = 'data/WIDERFace/WIDER_test/'
img_height = 224
img_width = 224
batch_size = 32

# Rescale pixel values to [0, 1]
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [26]:

img_height, img_width = 224, 224
train_dir = 'data/WIDERFace/WIDER_train/'
validation_dir = 'data/WIDERFace/WIDER_val/'
batch_size = 32
epochs = 10


import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# Define the dataset paths
# Convert the annotation files from the WIDERFace format to COCO format
# Split the dataset into training and validation sets

# Define the image size and batch size
img_size = 224
batch_size = 32

# Define the data augmentation settings
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

# Define the generators for the training and validation sets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

# Normalize the pixel values of the images to be between 0 and 1
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

# Fit the model to the training data and validate on the validation data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size)


Found 12880 images belonging to 61 classes.
Found 3226 images belonging to 61 classes.
Found 12880 images belonging to 61 classes.
Found 3226 images belonging to 61 classes.
Epoch 1/10
342/402 [========================>.....] - ETA: 2:36 - loss: 0.0901 - accuracy: 0.1096

In [ ]:
import matplotlib.pyplot as plt

# Get the training and validation loss and accuracy from history.history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plot the training and validation loss over the epochs
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy over the epochs
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
test_dir = 'data/WIDERFace/WIDER_test/'

# Define the generator for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)

print('Test accuracy:', test_acc)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Make predictions on the test set
test_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_size, img_size),
    batch_size=1,
    shuffle=False,
    class_mode='categorical')

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get a list of class names
class_names = list(train_generator.class_indices.keys())

# Plot some examples with their predicted class labels
num_examples = 10
plt.figure(figsize=(10, 10))
for i in range(num_examples):
    img, true_label = test_generator[i]
    predicted_label = predicted_classes[i]
    plt.subplot(5, 2, i+1)
    plt.imshow(img[0])
    plt.axis('off')
    plt.title("True: {} \nPredicted: {}".format(class_names[true_label[0].argmax()], class_names[predicted_label]))
plt.show()


In [ ]:
from keras.models import load_model
import cv2
import numpy as np

# Save the trained model as an h5 file
model.save('cnn_sdd_model.h5')

# Load the saved model
model = load_model('cnn_sdd_model.h5')

# Test the model on a new image
img = cv2.imread('./samples/1.jpg')
img = cv2.resize(img, (224, 224))
img = np.expand_dims(img, axis=0)
img = img / 255.0

prediction = model.predict(img)

# Plot the image with the predicted bounding box
import matplotlib.pyplot as plt

plt.imshow(cv2.cvtColor(cv2.imread('./samples/1.jpg'), cv2.COLOR_BGR2RGB))
plt.plot(prediction[0][:, 0], prediction[0][:, 1], 'bo', markersize=2)
plt.show()
